In [1]:
#This is going to be the Python simulation node. Currently just a code to plot the trajectory of the modeled car
#as calculated by the MPC node in Julia, and the path follower code used as another node

# importing libraries 
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
import numpy as np
from PIL import Image
import pandas as pd
import csv

# #open map image and resize to size of carpark
# image = Image.open('Map.png')
# # Read Images  
# new_image = image.resize((300, 200))
# new_image.save('Map_resize.png')
# plt.imshow(new_image)
img = mpimg.imread('Map.png')
new_image = plt.imshow(img)
#load in MPC generated data for X, Y, velocity and heading at each point
opened_file = open('Z.csv')
from csv import reader
read_file = reader(opened_file)
Z = list(read_file)
n=len(Z)
z_list = [list(map(float,i)) for i in Z]
z_list=np.array(z_list)

In [2]:
def plotting_func(z0):
    # Plot the trajectory with the car visualized simultaneously
    # Parameters of car
    lf = 1.738
    lr = 1.738
    width = 2
    # Assume the length of the wheel is 0.5
    lw = 0.5
    #figure
    # front
    x_f = z0[0]+lf*np.cos(z0[3])
    y_f = z0[1]+lf*np.sin(z0[3])
    # front left
    x_f_l = x_f-(width/2)*np.sin(z0[3])
    y_f_l = y_f+(width/2)*np.cos(z0[3])
    # front right
    x_f_r = x_f+(width/2)*np.sin(z0[3])
    y_f_r = y_f-(width/2)*np.cos(z0[3])
    # rear
    x_r = z0[0]-lr*np.cos(z0[3])
    y_r = z0[1]-lr*np.sin(z0[3])
    # rear left
    x_r_l = x_r-(width/2)*np.sin(z0[3])
    y_r_l = y_r+(width/2)*np.cos(z0[3])
    # rear right
    x_r_r = x_r+(width/2)*np.sin(z0[3])
    y_r_r = y_r-(width/2)*np.cos(z0[3])
    # draw the car

  
    h1=new_image.plot([x_f_l,x_f_r],[y_f_l,y_f_r],'b-')
    h2=new_image.plot([x_f_l,x_r_l],[y_f_l,y_r_l],'b-')
    h3=new_image.plot([x_r_l,x_r_r],[y_r_l,y_r_r],'b-')
    h4=new_image.plot([x_f_r,x_r_r],[y_f_r,y_r_r],'b-')
    pause(1)
    # calculate the wheel
    # left wheel
    w_f_l_x = x_f_l+(lw/2)*np.cos(z0[3]-u[1])
    w_r_l_x = x_f_l-(lw/2)*np.cos(z0[3]-u[1])
    w_f_l_y = y_f_l+(lw/2)*np.sin(z0[3]-u[1])
    w_r_l_y = y_f_l-(lw/2)*np.sin(z0[3]-u[1])
    # right wheel
    w_f_r_x = x_f_r+(lw/2)*np.cos(z0[3]-u[1])
    w_r_r_x = x_f_r-(lw/2)*np.cos(z0[3]-u[1])
    w_f_r_y = y_f_r+(lw/2)*np.sin(z0[3]-u[1])
    w_r_r_y = y_f_r-(lw/2)*np.sin(z0[3]-u[1])

    # Draw the wheel
    h5=new_image.plot([w_f_l_x,w_r_l_x],[w_f_l_y,w_r_l_y],'r-')  
    h6=new_image.plot([w_f_r_x,w_r_r_x],[w_f_r_y,w_r_r_y],'r-') 
    pause(1)
  
    new_image.plot(z_list[0,:], z_list[1,:])

In [3]:
for i in range(len(z_list)):
    z0=np.array([z_list[i,0],z_list[i,1], z_list[i,2], z_list[i,3]])
    plotting_func(z0)
    

AttributeError: 'AxesImage' object has no attribute 'plot'